In [ ]:
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import time
import datetime as dt
import os
import seaborn as sns
import scipy.stats as sts

import random
import string

In [ ]:
from nb_vars import FOLDER_OUTPUT_BASE, CUTOFF_SPIKES_HZ

## Load df of filtered peaks

In [ ]:
CONDITION_NAME = "pulse_duration"
CONDITION = "200"

folder_input =  f'{FOLDER_OUTPUT_BASE}/{CONDITION_NAME}_{CONDITION}'

In [ ]:
df_peaks = pl.read_csv(f'{folder_input}/df_peaks_full_{CONDITION_NAME}_{CONDITION}_freq_{CUTOFF_SPIKES_HZ}.csv').to_pandas()


In [ ]:
df_peaks

## Relative change in MFR

In [ ]:
df_MFR = df_peaks.groupby(['condition', 'treatment', 'well', 'replicate', 'electrode']).count()['time'] / (df_peaks['time'].max() - df_peaks['time'].min())
df_MFR = df_MFR.reset_index().sort_values(by=['well', 'electrode'])
df_MFR = df_MFR.rename(columns={'time': 'MFR'})
df_MFR = df_MFR.reset_index(drop=True)
df_MFR

In [ ]:
df_MFR_summary = df_MFR.pivot(index=['condition', 'well', 'replicate', 'electrode'], columns='treatment', values='MFR').reset_index()
df_MFR_summary = df_MFR_summary[['condition',	'well',	'replicate',	'electrode',	'PRE',	'LINKER',	'POST']]

### Change POST vs PRE, LINKER vs PRE and POST vs LINKER

In [ ]:
df_MFR_summary['POST_vs_PRE'] = 100 * df_MFR_summary['POST'] / df_MFR_summary['PRE']
df_MFR_summary['POST_vs_LINKER'] =  100 * df_MFR_summary['POST'] / df_MFR_summary['LINKER'] 
df_MFR_summary['LINKER_vs_PRE'] =  100 * df_MFR_summary['LINKER'] / df_MFR_summary['PRE']

df_MFR_summary.to_csv(f'{folder_input}/MFR_summary_{CONDITION_NAME}_{CONDITION}_freq_{CUTOFF_SPIKES_HZ}.csv', sep=';', index=None)
df_MFR_summary


In [ ]:
fig = plt.figure(figsize=(29,6))
sns.swarmplot(data = df_MFR_summary, x='condition', y='POST_vs_PRE', hue='replicate')
plt.plot([-0.5, len(df_MFR_summary['condition'].drop_duplicates()) - 0.5], [100, 100], c='#bc0000')

In [ ]:
fig = plt.figure(figsize=(29,6))
sns.boxplot(data = df_MFR_summary, x='condition', y='POST_vs_PRE')
plt.plot([-0.5, len(df_MFR_summary['condition'].drop_duplicates()) - 0.5], [100, 100], c='#bc0000')

In [ ]:
# %%script true

# GRAPHS TO CHECK IF THERES A BIAS BETWEEN THE RATIO AND THE ACTIVITY

sns.scatterplot(data = df_MFR_summary, x='PRE', y='POST_vs_PRE', label='PRE')
sns.scatterplot(data = df_MFR_summary, x='POST', y='POST_vs_PRE', label='POST')


In [ ]:
# %%script true


df_MFR_summary['mean_PRE_POST'] = (df_MFR_summary['PRE'] + df_MFR_summary['POST']) / 2

#create regplot
p = sns.regplot(data = df_MFR_summary, x='mean_PRE_POST', y='POST_vs_PRE')

#calculate slope and intercept of regression equation
slope, intercept, r, p, sterr = sts.linregress(x=p.get_lines()[0].get_xdata(),
                                                       y=p.get_lines()[0].get_ydata())

#display slope and intercept of regression equation
print(intercept, slope, p)
